## Load the textbook embeddings and instantiate the model

In [1]:
import pickle

In [2]:
def load_embeddings_to_pickle(file_path):
    with open(file_path,'rb') as f:
        embedding_with_metadata = pickle.load(f)
    return embedding_with_metadata


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')



#helper function to embed text chunks with metadata
def embed_text_chunks(textbook_chunk_metadata,model):
    for chunk in textbook_chunk_metadata:
        text_chunk = chunk['text_chunk']
        embedding = model.encode(text_chunk)
        chunk['embedding'] = embedding
    return textbook_chunk_metadata

In [13]:
combined_textbook_embed = load_embeddings_to_pickle('/Users/haridevaraj/Documents/Projects/steps_ai/Content_extraction_and_chunking_embed/combined_textbook_embedding_metadata.pkl')

In [8]:
# textbook1_embed = load_embeddings_to_pickle('/Users/haridevaraj/Documents/Projects/steps_ai/Content_extraction_and_chunking_embed/textbook1_embedding_metadata.pkl')
# textbook2_embed =load_embeddings_to_pickle('/Users/haridevaraj/Documents/Projects/steps_ai/Content_extraction_and_chunking_embed/textbook2_embedding_metadata.pkl')
# textbook3_embed =load_embeddings_to_pickle('/Users/haridevaraj/Documents/Projects/steps_ai/Content_extraction_and_chunking_embed/textbook3_embedding_metadata.pkl')

## RAPTOR indexing

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import 


In [ ]:
def cluster_embeddings(embeddings,n_components=10):
    gmm = GaussianMixture(n_components=n_components,covariance_type='full')
    gmm.fit(embeddings)
    clusters = gmm.predict_proba(embeddings)
    return clusters, gmm

def optimal_num_cluster(embeddings,max_cluster=30):
    max_clusters = min(max_clusters,len(embeddings))
    


In [15]:
embeddings = np.array([chunk['embedding']]for chunk in combined_textbook_embed)

In [19]:
len(combined_textbook_embed)

4497

In [ ]:
#reducing the cluster using UMAP dimensionality reduction

